# make a very simple LSTM model with keras

see how long this will take to run for 7t data with a not too complicated model and what kind of resources we are talking about - single run single subject..

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler
import pandas as pd
import nibabel as nb
import hrf_tools
from analysis import load_data_HCP_MMP

/opt/miniconda-latest/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
print(np.__version__)

1.19.5


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13014453756933303561
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7611585664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9254048372183658020
physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:02:00.0, compute capability: 7.5"
]


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

### load data

In [3]:
subject=100610
feature='as_scores'
n_movies=[1,2,3]
X,Y = load_data_HCP_MMP(subject,feature,n_movies)

X_test,Y_test = load_data_HCP_MMP(subject,feature,[4])

In [11]:
X = hrf_tools.apply_optimal_hrf_10hz(X,1)
X_test = hrf_tools.apply_optimal_hrf_10hz(X_test,1)

In [16]:
X.shape

(2385, 521)

In [30]:
# Batch size
batch_size = 64
time_steps = 10
input_units = 521
input_shape = (time_steps,input_units)

In [31]:
#https://keras.io/api/preprocessing/timeseries/
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(data = X, 
                                                     targets = Y, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)
dataset_test = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_test, 
                                                     targets = Y_test, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

### design a simple(?) lstm model

In [32]:


model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
#model.add(layers.Embedding(input_dim=1024, output_dim=1024))

#model.add(keras.Input(shape=input_shape)) #omit beacuse of bug
# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(input_units,batch_input_shape=(batch_size, time_steps, input_units)))
model.add(layers.Dense(521))
model.add(layers.Dense(521))
model.add(layers.Dense(360))

#model.add(layers.UpSampling1D(size=10))
#model.add(layers.Dense(17049))
#model.add(layers.Dense(128))
model.add(layers.Dense(360))
#model.add(layers.Dense(360))


model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (64, 521)                 2173612   
_________________________________________________________________
dense_12 (Dense)             (64, 521)                 271962    
_________________________________________________________________
dense_13 (Dense)             (64, 521)                 271962    
_________________________________________________________________
dense_14 (Dense)             (64, 360)                 187920    
_________________________________________________________________
dense_15 (Dense)             (64, 360)                 129960    
Total params: 3,035,416
Trainable params: 3,035,416
Non-trainable params: 0
_________________________________________________________________


In [33]:
# #dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(x_train,0), np.expand_dims(y_train,0)))
# dataset = tf.data.Dataset.from_tensor_slices((X_hrf, Y))

# dataset.element_spec

In [34]:


# # Buffer size to shuffle the dataset
# # (TF data is designed to work with possibly infinite sequences,
# # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# # it maintains a buffer in which it shuffles elements).
# BUFFER_SIZE = 10000

# dataset = (
#     dataset
#     .shuffle(BUFFER_SIZE)
#     .batch(batch_size, drop_remainder=True)
#     .prefetch(tf.data.experimental.AUTOTUNE))

# dataset

In [35]:
#simple_rnn = layers.SimpleRNN(18, return_sequences=True, return_state=True)

In [36]:
#whole_sequence_output, final_state = simple_rnn(np.expand_dims(x_train,0))

In [37]:
model.compile(
    loss=keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    optimizer="adam",
    metrics=["accuracy", 'mean_squared_error'],
)

In [38]:
history = model.fit(dataset,epochs = 20)

Epoch 1/20
37/38 [============================>.] - ETA: 0s - loss: 0.9501 - accuracy: 0.0022 - mean_squared_error: 0.9501

InvalidArgumentError:    Specified a list with shape [64,521] from a tensor with shape [8,521]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential_3/lstm_3/PartitionedCall]] [Op:__inference_train_function_4941]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
model.save('../outputs/model.pb')

In [ ]:
model = keras.models.load_model('../outputs/model.pb')

In [72]:
dataset_test = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_test_hrf, 
                                                     targets = Y_test, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

In [74]:
model.evaluate(dataset_test)

15/15 [==============================] - 0s 14ms/step - loss: 2109020.0000 - accuracy: 0.0000e+00 - mean_squared_error: 2109020.0000


[2109020.0, 0.0, 2109020.0]